# Anwendung 2: Topic Modeling

Eine weitere Anwendung von Vector-Space-Modellen ist das Topic Modeling. Es zielt auf die Identifikation von »topics« in einem Corpus. Die Berechnung dieser topics basiert dabei auf ihrer Verteilung in den Dokumenten des Corpus und damit letztlich auf Wort-Kookkurrenzen. Ob die ermittelten topics tatsächlich *Themen* im inhaltlichen Sinne, oder eher »Diskursstränge«, Wort-Cluster o.ä. sind, bleibt der inhaltlichen Interpretation überlassen.

Weiter kann an dieser Stelle nicht auf die Hintergründe des Topic Modeling eingegangen werden. Für die weitere Lektüre sei daher dieser Beitrag empfohlen:

Brett, Megan R. (2012): „Topic Modeling: A Basic Introduction“, Journal of Digital Humanities 2/1, http://journalofdigitalhumanities.org/2-1/topic-modeling-a-basic-introduction-by-megan-r-brett/.

Für das Topic Modeling muss der Text nur minimal aufbereitet werden. Im Gegensatz zu der `TextCorpus`-Klasse, die für die Keyword Extraction verwendet wurde, wurden aber zwei Details verändert, die die Qualität der erzeugten Topics verbessern:

* Für die Tokenisierung, also die Identifikation von Worten, wird nicht auf `TextBlob` zurückgegriffen, sondern der reguläre Ausdruck `\w+` verwendet. Durch die Begrenzung auf alphanumerische Zeichen werden etwa Satzzeichen wie Bindestriche oder Anführungszeichen automatisch aussortiert.
* Es wird ein Stoppwortfilter verwendet, der auf einer vorgegebenen Liste basiert.

Diese beiden Punkte sind in den Methoden `tokenize()` und `filter()` umgesetzt. Die Methode `get_texts()` wurde entsprechend angepasst. Zusätzlich sind noch zwei Details umgesetzt, die die Verarbeitungsgeschwindigkeit erhöhen: So wird die CSV-Tabelle nur einmal eingelesen, anstatt bei jedem Durchlauf neu geladen zu werden. Ebenso wird die Anzahl der Dokumente im Corpus, die für manche Berechnungen notwendig ist, zwischengespeichert. Diese beiden Punkte sind aber nicht zwingend erforderlich.

In [1]:
import codecs
import re

from gensim.corpora.textcorpus import TextCorpus
from textblob_de import TextBlobDE as TextBlob
from textblob_de import PatternParser
import pandas as pd

class CSVCorpus(TextCorpus):
    """Read corpus from a csv file."""

    def tokenize(self, text):
        words = re.findall(u'\w+', text.lower(), re.U)
        return words

    def filter(self, tokens, stopwords):
        return [token for token in tokens if not token in stopwords]

    def get_texts(self):
        with codecs.open('../Daten/stopwords.txt', encoding='utf-8') as stopwordfile:
            stopwords = [line.strip() for line in stopwordfile.readlines()]
        table = self.gettable()
        for text in table['text']:
            tokens = self.tokenize(text)
            yield self.filter(tokens, stopwords)

    def gettable(self):
        if not hasattr(self, 'table'):
            with self.getstream() as csvfile:
                self.table = pd.read_csv(csvfile, parse_dates=['date'], encoding='utf-8')
        return self.table

    def __len__(self):
        if not hasattr(self, 'length'):
            # Cache length
            self.length = len(self.gettable())
        return self.length

Alternativ kann auch auf die Verfahren der Lemmatisierung und Wortartenfilterung zurückgegriffen werden, die in den vorherigen Einheiten besprochen wurden. Dazu werden die beiden Methoden `tokenzie()` und `filter()` überschrieben. Damit dauert die Verarbeitung aber deutlich länger, und es ist unklar, ob die Qualität der Analyse dadurch zwingend steigt. Für ein vergleichendes Experiment kann diese Version aber verwendet werden.

In [2]:
from string import punctuation
from collections import namedtuple

class LemmatizedCSVCorpus(CSVCorpus):
    """Read corpus from a csv file."""

    def tokenize(self, text):
        text = text.replace(u'\xa0', u' ')  # Ersetze "non-breaking space" durch normales Leerzeichen
        text = re.sub(u'[„“”‚‘’–]', '', text, re.U)  # Entferne Anführungzeichen und Gedankenstriche
        blob = TextBlob(text, parser=PatternParser(lemmata=True))
        parse = blob.parse()
        fieldnames = [tag.replace('-', '_') for tag in parse.tags]
        Token = namedtuple('Token', fieldnames)
        tokens = [Token(*token.split('/', 4)) for token in parse.split(' ')]
        return tokens

    def filter(self, tokens, stopwords):
        result = []
        for token in tokens:
            pos = token.part_of_speech[0:2]
            word = token.word
            if not word.lower() in stopwords and not word in punctuation:
                if pos == 'NN':
                    result.append(token.lemma.title())
                else:
                    result.append(token.lemma)
        return result

In [3]:
corpus = CSVCorpus('../Daten/reden.csv')
len(corpus.dictionary)

48968

In der nicht lemmatisierten Form enthält das Corpus nun also 48968 unterschiedliche Wörter. Besonders häufige ebenso wie besonders seltene Wörter können dabei die Analyse negativ beeinflussen. Gensim stellt eine Methode bereit, mit der das Corpus um diese Extremwerte bereinigt werden kann. (Erst im zweiten Schritt `compactify()` werden dann die gelöschten Extremwerte tatsächlich aus dem Corpus entfernt.)

In [4]:
corpus.dictionary.filter_extremes()
corpus.dictionary.compactify()
len(corpus.dictionary)

10346

Nach dem Filtern bleiben noch 10346 Einträge übrig.

Gensim enthält eine Implementierung des Topic-Modeling-Verfahrens »LDA« (neben anderen). Diese Version ist dabei für sehr große Corpora optimiert, die resultierenden Topics sind aber leider oft nicht sehr leicht zu interpretieren. Daher soll hier die Implementierung aus dem Python-Paket »lda« verwendet werden. Sie arbeitet nicht direkt mit dem TextCorpus-Format von gensim, kann aber eine *sparse matrix* einlesen, die mit gensim erzeugt wird.

In [5]:
from gensim.matutils import corpus2csc

corpus_matrix = corpus2csc(corpus)
corpus_matrix.shape

(10346, 614)

In [6]:
corpus_matrix = corpus_matrix.transpose()
corpus_matrix.shape

(614, 10346)

Ähnlich wie bei der Keyword Extraction wird hier im ersten Schritt ein Modell erzeugt, das vor allem bestimmte Parameter für die Berechnung speichert. Der wichtigste ist hierbei die Anzahl der Topics. Diese muss vorgegeben werden und kann nicht vom Algorithmus selbst bestimmt werden. 20 ist oft ein guter Ausgangswert, man sollte aber mit verschiedenen Werten experimentieren und die Ergebnisse vergleichen.

In [7]:
import lda
lda_model = lda.LDA(n_topics=20, n_iter=500, random_state=1)
lda_model.fit(corpus_matrix)

<lda.lda.LDA instance at 0x7f48a410a200>

*Hinweis:* Um die Ergebnisse etwas übersichtlicher darzustellen, ist eine gewisse Formatierung der Ausgabe nützlich. Dies könnte etwa mit HTML realisiert werden. Etwas einfacher ist das minimalistische Textformat »Markdown«, das etwa Zeilen mit vorangestellten Sternchen `*` in Aufzählungslisten umwandelt. Um dies einfacher zu nutzen, wird hier eine kleine Hilfsfunktion definiert. Zu Details siehe die [Syntax-Beschreibung](http://www.daringfireball.net/projects/markdown/syntax).

In [8]:
from IPython.nbconvert.filters.markdown import markdown2html

class MD(unicode):
    def _repr_html_(self):
        return markdown2html(self)

MD(u'Das ist ein **Test!**')

/home/frederik/Dropbox/Dokumente/Lehrveranstaltungen/LA_Luzern/Python/pyenv/local/lib/python2.7/site-packages/IPython/nbconvert/filters/markdown.py:78: UserWarning: Node.js 0.9.12 or later wasn't found.
Nbconvert will try to use Pandoc instead.
  "Nbconvert will try to use Pandoc instead.")


u'Das ist ein **Test!**'

Das Ergebnisformat von LDA sind Matrizen im numpy-Format. Diese sind sehr effizient und bieten eine Reihe von Berechnungsmöglichkeiten, sie sind jedoch auf den ersten Blick nicht ganz leicht zu verstehen. Der folgende Code wurde aus der Dokumentation des lda-Pakets übernomme und ein wenig angepasst. Für den Moment soll die Beschreibung ausreichen, dass hierüber für jedes Topic die einflussreichsten 20 Wörter ausgegeben werden.

In [9]:
import numpy as np
vocab = [corpus.dictionary[i] for i in range(len(corpus.dictionary))]
topic_word = lda_model.topic_word_  # model.components_ also works
n_top_words = 20
topics = [np.array(vocab)[np.argsort(topic_dist)][:-n_top_words:-1]
          for topic_dist in topic_word]
result = u''
for i, topic_words in enumerate(topics):
    result += u'* **Topic {}:** {}\n'.format(i, ' '.join(topic_words))
MD(result)

u'* **Topic 0:** politik liebe weise blick weit sinne leisten zeigt gesellschaft erfolg bedeutung stelle genau deutlich vergangenen recht wichtiger thema wichtige\n* **Topic 1:** erinnerung opfer gedenken geschichte vertreibung vertriebenen flucht erinnern aufarbeitung nationalsozialismus gedenkst\xe4tte ort juden aufgabe verbrechen nationalsozialistischen haus ns vers\xf6hnung\n* **Topic 2:** kultur kunst berlin ausstellung stiftung bund museum kulturellen museen haus k\xfcnstler kulturelle gesprochene anrede bundes wort geschichte einrichtungen 1\n* **Topic 3:** krise wachstum prozent wettbewerbsf\xe4higkeit wirtschaft wirtschaftskrise union augenblick internationalen energien europ\xe4ische g20 staaten finanz vertrauen euroraum griechenland situation stark\n* **Topic 4:** film filme welle berlinale kinos urheber kino kommission verleihung gesprochene herzlich kreative musik films wort insbesondere f\xf6rderung anrede jury\n* **Topic 5:** energien nachhaltigkeit erneuerbaren landwirtschaft klimaschutz internationalen prozent ziel thema erreichen nachhaltige co2 ziele vereinten nationen umwelt wohlstand grad ressourcen\n* **Topic 6:** gesellschaft familie kinder frauen familien beruf eltern wandel bildung lernen thema junge alter schule kommunen chancen kindern liebe zusammenhalt\n* **Topic 7:** freiheit geschichte einheit deutsch demokratie berlin 20 staaten politischen weltkrieg deutschlands kraft israel berliner frieden freundschaft generation toleranz staat\n* **Topic 8:** internet digitalen kultur digitalisierung digitale vielfalt \xf6ffentlich schutz rahmenbedingungen initiative kreativen eu urheberrecht geistigen rechtlichen kreativwirtschaft eigentums netz privaten\n* **Topic 9:** sicherheit bundeswehr afghanistan herausforderungen soldaten einsatz staaten nato internationalen iran l\xf6sung region afrika frieden internationale genauso partnern terrorismus politischen\n* **Topic 10:** unternehmen bereich forschung wirtschaft thema industrie investitionen m\xf6glichkeiten rahmenbedingungen entwickeln bereichen hierbei region it denke bereiche infrastruktur insgesamt freue\n* **Topic 11:** prozent unternehmen thema bereich milliarden n\xe4chsten 40 zahl forschung 50 ausbildung situation jungen punkt rente arbeitnehmerinnen demografischen kommunen legislaturperiode\n* **Topic 12:** union europ\xe4ische mitgliedstaaten europas gemeinsame gemeinsamen frankreich w\xe4hrungsunion wirtschafts europ\xe4er einigung krise eu parlament w\xe4hrung gl\xfcck vertrag kontinent kommission\n* **Topic 13:** richtig sozusagen aufgabe schaffen dingen \xfcberhaupt gesprochen politik denken worden sprechen sozialen bekommen diskussion schritt f\xfchren 20 stellen probleme\n* **Topic 14:** ddr geschichte helmut kohl ost diktatur wiedervereinigung 1989 ehemaligen bundesrepublik sed teilung revolution 20 b\xfcrger west 1990 de mauer\n* **Topic 15:** bund 2 mittel philipp verf\xfcgung schritt wirtschaft legislaturperiode r\xf6sler 000 5 bayern ziel \xfcbrigen bildung ausgaben haushalt investition einf\xfchrung\n* **Topic 16:** banken notwendig zusammenarbeit international regulierung recht risiken richtig hierbei meinung punkt regelungen sparkassen gefahr halte bezug form kommission transparenz\n* **Topic 17:** wissenschaft bildung universit\xe4t professor chemie max wissenschaftlichen humboldt wissenschaften universit\xe4ten f\xf6rdern kooperation petra grundlagenforschung forschungseinrichtungen studenten hochschulen jungen institut\n* **Topic 18:** medien sachsen zeitungen zeitung anhalt journalisten presse informationen journalismus demokratie verlag interesse leser preistr\xe4ger grosso kinder haseloff meinungsfreiheit demokratischen\n* **Topic 19:** china zusammenarbeit beziehungen deutsch russland t\xfcrkei wirtschaft kooperation unternehmen mecklenburg vorpommern chinesischen wirtschaftliche bulgarien wirtschaftlichen premierminister chinesische delegation interesse\n'

Diese Topics sind zunächst einmal probabilistisch identifizierte Wort-Cluster. Bei genauerer Betrachtung lässt sich aber für die meisten Topics ein Eindruck gewinnen, welches Thema die ausgegebenen Worte umreißen. Topic 1 etwa beschreibt die Auseinandersetzung mit dem Nationalsozialismus, Topic 2 dagegen den Bereich Kunst und Kultur. Topic 0 dagegen lässt sich auf den ersten Blick weniger leicht interpretieren.

Neben den Topics selbst gibt LDA auch eine Zuordnung von Topics zu Dokumenten aus. Es lassen sich also auch für jedes Dokument die relevantesten Topics ausgeben, die das Dokument beschreiben. Die ist ähnlich wie bei der Keyword Extraction, nur dass ganze Topics und nicht einzelne Schlüsselwörter zur Beschreibung herangezogen werden.

In [10]:
data = pd.read_csv("../Daten/reden.csv", parse_dates=['date'], encoding='utf-8')
titles = data['title']

doc_topic = lda_model.doc_topic_
result = u''
for i in range(10):
    result += u'\n\n**{}**\n\n'.format(titles[i])
    for topic in doc_topic[i].argsort()[:-4:-1]:
        result += u' * _Topic {}:_ {}\n'.format(topic, ' '.join(topics[topic]))
MD(result)

u'\n\n**Laudatio auf R\xfcdiger Safranski**\n\n * _Topic 0:_ politik liebe weise blick weit sinne leisten zeigt gesellschaft erfolg bedeutung stelle genau deutlich vergangenen recht wichtiger thema wichtige\n * _Topic 7:_ freiheit geschichte einheit deutsch demokratie berlin 20 staaten politischen weltkrieg deutschlands kraft israel berliner frieden freundschaft generation toleranz staat\n * _Topic 2:_ kultur kunst berlin ausstellung stiftung bund museum kulturellen museen haus k\xfcnstler kulturelle gesprochene anrede bundes wort geschichte einrichtungen 1\n\n\n**Rede von Bundeskanzlerin Merkel anl\xe4sslich der Pr\xe4sentation des deutsch-chinesischen Kooperationsprojekts zur Ladeinfrastruktur f\xfcr Elektrofahrzeuge am 8. Juli 2014**\n\n * _Topic 10:_ unternehmen bereich forschung wirtschaft thema industrie investitionen m\xf6glichkeiten rahmenbedingungen entwickeln bereichen hierbei region it denke bereiche infrastruktur insgesamt freue\n * _Topic 0:_ politik liebe weise blick weit sinne leisten zeigt gesellschaft erfolg bedeutung stelle genau deutlich vergangenen recht wichtiger thema wichtige\n * _Topic 19:_ china zusammenarbeit beziehungen deutsch russland t\xfcrkei wirtschaft kooperation unternehmen mecklenburg vorpommern chinesischen wirtschaftliche bulgarien wirtschaftlichen premierminister chinesische delegation interesse\n\n\n**Rede von Bundeskanzlerin Merkel anl\xe4sslich des Besuchs der Tsinghua - Universit\xe4t am 8. Juli 2014**\n\n * _Topic 0:_ politik liebe weise blick weit sinne leisten zeigt gesellschaft erfolg bedeutung stelle genau deutlich vergangenen recht wichtiger thema wichtige\n * _Topic 5:_ energien nachhaltigkeit erneuerbaren landwirtschaft klimaschutz internationalen prozent ziel thema erreichen nachhaltige co2 ziele vereinten nationen umwelt wohlstand grad ressourcen\n * _Topic 19:_ china zusammenarbeit beziehungen deutsch russland t\xfcrkei wirtschaft kooperation unternehmen mecklenburg vorpommern chinesischen wirtschaftliche bulgarien wirtschaftlichen premierminister chinesische delegation interesse\n\n\n**Kulturstaatsministerin Monika Gr\xfctters zur Er\xf6ffnung der Sonderausstellung\r\n"Flughafen Berlin-Tempelhof - Die amerikanische Geschichte"**\n\n * _Topic 0:_ politik liebe weise blick weit sinne leisten zeigt gesellschaft erfolg bedeutung stelle genau deutlich vergangenen recht wichtiger thema wichtige\n * _Topic 7:_ freiheit geschichte einheit deutsch demokratie berlin 20 staaten politischen weltkrieg deutschlands kraft israel berliner frieden freundschaft generation toleranz staat\n * _Topic 2:_ kultur kunst berlin ausstellung stiftung bund museum kulturellen museen haus k\xfcnstler kulturelle gesprochene anrede bundes wort geschichte einrichtungen 1\n\n\n**Rede von Bundeskanzlerin Merkel anl\xe4sslich der Taufe des Tiefseeforschungsschiffs \u201eSonne\u201c am 11. Juli 2014 **\n\n * _Topic 0:_ politik liebe weise blick weit sinne leisten zeigt gesellschaft erfolg bedeutung stelle genau deutlich vergangenen recht wichtiger thema wichtige\n * _Topic 17:_ wissenschaft bildung universit\xe4t professor chemie max wissenschaftlichen humboldt wissenschaften universit\xe4ten f\xf6rdern kooperation petra grundlagenforschung forschungseinrichtungen studenten hochschulen jungen institut\n * _Topic 5:_ energien nachhaltigkeit erneuerbaren landwirtschaft klimaschutz internationalen prozent ziel thema erreichen nachhaltige co2 ziele vereinten nationen umwelt wohlstand grad ressourcen\n\n\n**Rede von Bundeskanzlerin Merkel anl\xe4sslich des 5. Petersberger Klimadialogs am 14. Juli 2014**\n\n * _Topic 5:_ energien nachhaltigkeit erneuerbaren landwirtschaft klimaschutz internationalen prozent ziel thema erreichen nachhaltige co2 ziele vereinten nationen umwelt wohlstand grad ressourcen\n * _Topic 0:_ politik liebe weise blick weit sinne leisten zeigt gesellschaft erfolg bedeutung stelle genau deutlich vergangenen recht wichtiger thema wichtige\n * _Topic 3:_ krise wachstum prozent wettbewerbsf\xe4higk

Dabei ist zunächst auffällig, dass Topic 0 immer in den Top 3 auftaucht. Gemeinsam mit seiner inhaltlichen Vagheit ergibt sich der Eindruck, dass es sich weitgehend um ein Residual-Topic handelt, das relativ unspezifische, aber regelmäßig auftretenden Wörter gruppiert. Die anderen Topics geben dagegen einen relativ guten Einblick in die inhaltlichen Schwerpunkte der Texte.

Um später erneut auf die Ergebnisse zurückgreifen zu können, wird das berechnete Modell gespeichert.

In [11]:
from cPickle import dump

with open('topicmodel20.pickle', 'wb') as picklefile:
    dump(lda_model, picklefile)